In [27]:
import os
import numpy as np
import pandas as pd # Get Relevant information from CSEM# Get Relevant information from MBT file file

# Get Relevant information from CSEM file

In [40]:
CSEM_Complete = pd.read_csv('CSEM_Master_RT.csv')
CSEM_Complete['Catalog Nbr'] = CSEM_Complete['Post Course'].fillna(CSEM_Complete['Pre Course']).astype(str).apply(lambda x: x[4:])
CSEM_Complete['NetID'] = CSEM_Complete['NetID'].str.replace(' ', '')
# Merge with Registrar Data
df = CSEM_Complete[['NetID', 'PreScore', 'PostScore', 'Semester', 'Catalog Nbr']].rename(columns = {'PreScore':'PreScores', 
                                                                                                    'PostScore':'PostScores', 
                                                                                                    'NetID':'Netid',
                                                                                                    'Semester':'Academic Term Sdescr'})

,Netid,PreScores,PostScores,Academic Term Sdescr,Catalog Nbr
0,yw458,9.0,22.0,2017FA,2213
1,qmh4,17.0,23.0,2017FA,2213
2,vek9,14.0,NaN,2017FA,2213
3,jds462,7.0,NaN,2017FA,2213
4,psk92,13.0,19.0,2017FA,2213


# Merge with Registrar Data

In [41]:
df_Registrar = pd.read_excel('Registrar_Data.xls').drop(columns = ['ACT | Combined English/Writing', 'ACT | English', 
                                                                   'ACT | Reading', 'ACT | Writing',
                                                                   'ACT | Writing Subject Score 9/15', 'SAT I | Critical Reading', 
                                                                   'SAT I | Read/Writing Sect Score', 'SAT I | Writing Score'])
df_Registrar[['Catalog Nbr', 'Employee Id']] = df_Registrar[['Catalog Nbr', 'Employee Id']].astype(str)
df_Registrar['Academic_Term_Number'] = df_Registrar['Academic Term Sdescr'].map({'2017FA':1, '2018SP':2, '2018FA':3, '2019SP':4})
df_Registrar = df_Registrar.sort_values(by = 'Academic_Term_Number').drop_duplicates(subset = ['Employee Id', 
                                                                                               'Catalog Nbr'], keep = 'last')

df_merged = pd.merge(df, df_Registrar, on = ['Netid', 'Academic Term Sdescr', 'Catalog Nbr'], 
                     how = 'inner').drop_duplicates(subset = ['Employee Id', 'Academic Term Sdescr'], 
                                                    keep = 'last').reset_index(drop = True)

index_vals = [tuple(v) for v in df_merged[['Employee Id', 'Academic Term Sdescr']].values]
filtered = df_Registrar.loc[(df_Registrar['Academic Term Sdescr'] + 
                             df_Registrar['Catalog Nbr']).isin(df_merged['Academic Term Sdescr'] + 
                                                               df_merged['Catalog Nbr'])].set_index(['Employee Id',
                                                                                                     'Academic Term Sdescr'])
filtered_out = filtered.loc[~filtered.index.isin(index_vals)].reset_index()

df_out = pd.concat([df_merged, filtered_out]).reset_index(drop = True)
df_out = df_out.loc[:, list(df_Registrar.columns) + ['PreScores', 'PostScores']]
df_out['Assessment'] = 'CSEM'
df_out.to_csv('CSEMpRegistrar.csv', index = False)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

